In [1]:
from IPython.display import display, HTML, Markdown
import os
import yaml
with open("../../../mkdocs.yml", "r") as f:
    load_config = yaml.safe_load(f)
repo_url = load_config["repo_url"].replace("https://github.com/", "")
binder_url = load_config["binder_url"]
relative_file_path = "integrations/Pytorch/basic_tutorial.ipynb"
display(HTML(f"""<a target="_blank" href="https://colab.research.google.com/github/{repo_url}/{relative_file_path}">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>"""))
display(Markdown("[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/SubhadityaMukherjee/openml_docs/HEAD?labpath=Scikit-learn%2Fdatasets_tutorial)"))

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/SubhadityaMukherjee/openml_docs/HEAD?labpath=Scikit-learn%2Fdatasets_tutorial)

In [1]:
!pip install openml-pytorch


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


# PyTorch sequential classification model example
An example of a sequential network that classifies digit images used as an OpenML flow.
We use sub networks here in order to show the that network hierarchies can be achieved with ease.

In [2]:
import torch.optim

import openml
import openml_pytorch
import openml_pytorch.layers
import openml_pytorch.config
import logging
import torch.nn as nn
import torch.nn.functional as F

from openml import OpenMLTask
import torchvision.models as models

In [3]:
# Enable logging in order to observe the progress while running the example.
openml.config.logger.setLevel(logging.DEBUG)
openml_pytorch.config.logger.setLevel(logging.DEBUG)

In [4]:
# Load the pre-trained ResNet model
model = models.resnet18(pretrained=True, progress=True)

# Modify the last fully connected layer to the required number of classes
num_classes = 20 # For the dataset we are using
in_features = model.fc.in_features
model.fc = nn.Linear(in_features, num_classes)

/Users/eragon/.pyenv/versions/3.9.19/envs/openml/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/eragon/.pyenv/versions/3.9.19/envs/openml/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [5]:

# Optional: If you're fine-tuning, you may want to freeze the pre-trained layers
for param in model.parameters():
    param.requires_grad = False

# If you want to train the last layer only (the newly added layer)
for param in model.fc.parameters():
    param.requires_grad = True

In [6]:
# Setting an appropriate optimizer 

def custom_optimizer_gen(model: torch.nn.Module, task: OpenMLTask) -> torch.optim.Optimizer:
    return torch.optim.Adam(model.fc.parameters())

openml_pytorch.config.optimizer_gen = custom_optimizer_gen

In [7]:
# Download the OpenML task for the Meta_Album_PNU_Micro dataset.
task = openml.tasks.get_task(361152)

############################################################################
# Run the model on the task (requires an API key).m
run = openml.runs.run_model_on_task(model, task, avoid_duplicate_runs=False)

/Users/eragon/Documents/CODE/Github/openml-pytorch/openml_pytorch/extension.py:154: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'encoded_labels'] = label_encoder.transform(y)
/Users/eragon/Documents/CODE/Github/openml-pytorch/openml_pytorch/extension.py:154: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'encoded_labels'] = label_encoder.transform(y)
/Users/eragon/Documents/CODE/Github/openml-pytorch/openml_pytorch/extension.py:154: SettingWithCopyWarning: 
A value is trying to be set o

In [15]:
import openml

In [26]:
print(run.flow)

OpenML Flow
Flow Name.......: torch.nn.ResNet.73f8a33b44a6743
Flow Description: Automatically created pytorch flow.
Dependencies....: torch==2.1.2
numpy>=1.6.1
scipy>=0.9


In [ ]:

# Publish the experiment on OpenML (optional, requires an API key).
run.publish()

print('URL for run: %s/run/%d' % (openml.config.server, run.run_id))
